## Tools And Libs

By: Alex Comerford (alexanderjcomerford@gmail.com)

In this notebook we will be going over a bunch of tools and libraries commonly used in scala to do actual useful work for common software tasks.

## Multi line string tokenization

A common problem in NLP is tokenization of text. In this next cell we will implement a multiline word tokenizer extractor to parse multi line strings into a list of words, then count those words in a table.

In [1]:
object MultiLineTokenize {
    def apply(s: String): Any = 
        s.split("\n")
         .toList
         .map(_.trim)
         .filter(_ != "")
         .map(_.split(" ").toList)
}

def flatten(e: Any): List[Any] = {
    e match {
        case Nil => Nil
        case head :: rest => flatten(head) ++ flatten(rest)
        case v => List(v)
    }
}

val input = """
This is a thing
sentences are yay
I like potatoes
They are much yum
"""
flatten(MultiLineTokenize(input)).groupBy(identity).mapValues(_.size)

## Writing and reading to a file

In this next cell we will go over a fundamental programming exercise, reading and writing to a file.

In [48]:
import java.io._
import scala.io.Source

val filename = "file.txt"
val file = new File(filename)

// Use java bufferwriter to write text to file
val bw = new BufferedWriter(new FileWriter(file))
bw.write(
"""
potato
tomato
apple
""")
bw.close()

// print contents of file
for (line <- Source.fromFile(filename).getLines) {
    println(line)
}

// create file monad 
implicit class FileMonads(f: File) {
  def check = if (f.exists) Some(f) else None
  def remove = if (f.delete()) Some(f) else None
}

// use match case to delete file
file check match {
    case Some(a) => {
        file remove;
        println(s"File $a deleted!")
    }
    case _ => {
        println("File Not found")
    }
}


potato
tomato
apple
File file.txt deleted!


null

## Executing system commands

In [59]:
import sys.process._

println("ls -al".!!)

// Multiple commands piping
val procs = ("ps a" #| "wc -l").!!.trim
println(s"procs = $procs")

// multiple command piping in shell
val files = Seq("/bin/sh", "-c", "ls | grep .ipynb").!!
println(files)

total 120
drwxrwxr-x. 3 beakerx beakerx  4096 Sep 24 23:53 .
drwxrwxr-x. 4 beakerx beakerx  4096 Sep 19 21:18 ..
-rw-r--r--. 1 root    root    70812 Sep 19 03:03 01_Basics.ipynb
-rw-r--r--. 1 root    root    15484 Sep 20 03:59 02_Algorithms.ipynb
-rw-r--r--. 1 root    root     6842 Sep 24 23:45 03_Collections.ipynb
-rw-r--r--. 1 root    root     9484 Sep 24 23:53 04_ToolsAndLibs.ipynb
drwxr-xr-x. 2 root    root     4096 Sep 24 19:01 .ipynb_checkpoints

procs = 2
01_Basics.ipynb
02_Algorithms.ipynb
03_Collections.ipynb
04_ToolsAndLibs.ipynb



null

## Concurrency!

This is the section I am most excited about. Scala is very well known for its concurrency model. Scala uses the actor model unlike golang CSP concurrency or others

We will be using the `akka` framework for these next examples

## SBT

In the rest of this notebook we will be using sbt and other libraries to experiment around

In [7]:
%%bash
echo "deb https://dl.bintray.com/sbt/debian /" | sudo tee -a /etc/apt/sources.list.d/sbt.list
sudo apt-key adv --keyserver hkp://keyserver.ubuntu.com:80 --recv 2EE0EA64E40A89B84B2DF73499E82A75642AC823
sudo apt-get update
sudo apt-get install sbt

deb https://dl.bintray.com/sbt/debian /
Executing: /tmp/tmp.YgKWK4WB1j/gpg.1.sh --keyserver
hkp://keyserver.ubuntu.com:80
--recv
2EE0EA64E40A89B84B2DF73499E82A75642AC823
gpg: requesting key 642AC823 from hkp server keyserver.ubuntu.com
gpg: key 642AC823: public key "sbt build tool <scalasbt@gmail.com>" imported
gpg: Total number processed: 1
gpg:               imported: 1  (RSA: 1)
Hit:1 http://archive.ubuntu.com/ubuntu xenial InRelease
Get:2 http://archive.ubuntu.com/ubuntu xenial-updates InRelease [109 kB]
Get:3 http://archive.ubuntu.com/ubuntu xenial-backports InRelease [107 kB]
Get:4 https://dl.yarnpkg.com/debian stable InRelease [13.3 kB]
Get:5 http://archive.ubuntu.com/ubuntu xenial-updates/universe Sources [280 kB]
Ign:6 https://dl.bintray.com/sbt/debian  InRelease
Get:7 http://security.ubuntu.com/ubuntu xenial-security InRelease [107 kB]
Get:8 https://dl.bintray.com/sbt/debian  Release [815 B]
Get:9 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 Packages [1,097 kB]
